Import library boto3 (library AWS SDK yang kompatibel dengan MinIO)

In [21]:
import boto3
import pandas as pd

Setup Path File Parquet

In [22]:
localPath = {
    'jakarta': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah/cleanedData/dapodik/dataYangSudahDiKategorisasi.csv',
    'bekasi': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_2/matchData/revisi/part2/hasilAkhir/Bekasi.csv',
    'depok': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_2/matchData/revisi/part2/hasilAkhir/Depok.csv',
    'balikpapan': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_3/afterCategorized/bekasiMatchCategorized.csv',
    'makassar': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_3/afterCategorized/makassarMatchCategorized.csv',
    'palembang': r'C:/Users/Asus/Downloads/Data Andalan Utama (Intern)/Exercise_Data_Sekolah_3/afterCategorized/palembangMatchCategorized.csv'
}

minioObjectName = {
    'jakarta': 'staging/parquet/boto3/JawaBarat/jakarta.parquet',
    'bekasi': 'staging/parquet/boto3/JawaBarat/bekasi.parquet',
    'depok': 'staging/parquet/boto3/JawaBarat/depok.parquet',
    'balikpapan': 'staging/parquet/boto3/kalimantanTimur/balikpapan.parquet',
    'makassar': 'staging/parquet/boto3/sulawesiSelatan/makassar.parquet',
    'palembang': 'staging/parquet/boto3/sumateraSelatan/palembang.parquet'
}

targetFilePath = {
    'jakarta': 'parquetLocaltemp/jakarta.parquet',
    'bekasi': 'parquetLocaltemp/bekasi.parquet',
    'depok': 'parquetLocaltemp/depok.parquet',
    'balikpapan': 'parquetLocaltemp/balikpapan.parquet',
    'makassar': 'parquetLocaltemp/makassar.parquet',
    'palembang': 'parquetLocaltemp/palembang.parquet'
}

In [23]:
dfJakarta = pd.read_csv(localPath['jakarta'], sep=';')
dfBekasi = pd.read_csv(localPath['bekasi'], sep=';')
dfDepok = pd.read_csv(localPath['depok'], sep=';')
dfBalikpapan = pd.read_csv(localPath['balikpapan'], sep=';')
dfMakassar = pd.read_csv(localPath['makassar'], sep=';')
dfPalembang = pd.read_csv(localPath['palembang'], sep=';')

In [24]:
dfJakarta.to_parquet(targetFilePath['jakarta'])
dfBekasi.to_parquet(targetFilePath['bekasi'])
dfDepok.to_parquet(targetFilePath['depok'])
dfBalikpapan.to_parquet(targetFilePath['balikpapan'])
dfMakassar.to_parquet(targetFilePath['makassar'])
dfPalembang.to_parquet(targetFilePath['palembang'])

Setup Kredensial MinIO

In [25]:
minioAuth = {
    'accessKey': 'minioadmin',
    'secretKey': 'miniopassword',
    'bucket': 'data-dapodik-kemendikdasmen',
    'endpoint': 'http://localhost:9000'
}

Inisialisasi Koneksi (Resource dan Client)

In [26]:
# Menggunakan Resource untuk fungsi level tinggi (seperti upload file)
s3Resource = boto3.resource(
    's3',
    endpoint_url = minioAuth['endpoint'],
    aws_access_key_id = minioAuth['accessKey'],
    aws_secret_access_key = minioAuth['secretKey'],
    verify = False # Wajib: Karena MinIO default di Docker itu http / tidak menggunakan SSL (https)
)

# Menggunakan Client untuk fungsi level rendah (seperti cek/buat bucket)
s3_client = boto3.client(
    's3',
    endpoint_url = minioAuth['endpoint'],
    aws_access_key_id = minioAuth['accessKey'],
    aws_secret_access_key= minioAuth['secretKey'],
    verify = False
)

Fungsi untuk memastikan bucket ada

In [27]:
def checkBucketIfExists(bucketName):
    try:
        
        # Jika sudah ada
        s3_client.head_bucket(Bucket = bucketName)
        print(f"Bucket '{bucketName}' sudah ada.")
        
    except Exception as e:
        
        # Jika bucket tidak ditemukan, buat bucket baru
        if '404' in str(e):
             s3_client.create_bucket(Bucket = bucketName)
             print(f"Bucket '{bucketName}' berhasil dibuat.")
             
        # Jika error bukan 404 (artinya ada kesalahan lain, biasanya salah endpoint api)
        else:
            print(f"Terjadi kesalahan saat memeriksa/membuat bucket: {e}")

checkBucketIfExists(minioAuth['bucket'])

Bucket 'data-dapodik-kemendikdasmen' sudah ada.


In [28]:
# Upload file cuaca
s3Resource.Bucket(minioAuth['bucket']).upload_file(
    targetFilePath['jakarta'],
    minioObjectName['jakarta']
    )
s3Resource.Bucket(minioAuth['bucket']).upload_file(
    targetFilePath['bekasi'],
    minioObjectName['bekasi']
    )
s3Resource.Bucket(minioAuth['bucket']).upload_file(
    targetFilePath['depok'],
    minioObjectName['depok']
    )
s3Resource.Bucket(minioAuth['bucket']).upload_file(
    targetFilePath['balikpapan'],
    minioObjectName['balikpapan']
    )
s3Resource.Bucket(minioAuth['bucket']).upload_file(
    targetFilePath['makassar'],
    minioObjectName['makassar']
    )
s3Resource.Bucket(minioAuth['bucket']).upload_file(
    targetFilePath['palembang'],
    minioObjectName['palembang']
    )

print("Berhasil mengunggah ke MinIO ✅")

Berhasil mengunggah ke MinIO ✅
